<a href="https://colab.research.google.com/github/MonkeyWrenchGang/MGTPython/blob/main/module_3/3_1_crunching_w_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crunching Data w. Pandas


---

Pandas is our library for data manipulation and analysis. It provides a variety of tools for "crunching" or "aggregating" data.

The most common method for aggregating data in Pandas is the `.groupby()` and `.agg()` or `.aggregate()` functions (agg is short for aggregate). These function allows you to group your data by one or more columns, and then perform a variety of aggregation operations on the groups. Some common aggregation operations include:

- mean(): calculates the mean of each group
- sum(): calculates the sum of each group
- count(): counts the number of non-NA/null values in each group
- min(): finds the minimum value in each group
- max(): finds the maximum value in each group

The `agg()` function allows us to perform multiple aggregation operations at once.

To get started cruncing we'll first  load our data into a DataFrame, explore the dataframe, and then use the groupby and agg functions to analyze the data. 


---


In this tutorial we'll perform the following
1. import the data
2. analyze missing values 
  - by row
  - by column 
3. aggregate 
  - single column 
  - multiple columns 
4. groupby and aggregate 
  - single column groupby single aggregate 
  - single column groupby multiple aggregates 
  - multiple column groupby single & multiple aggregate 
5. combining with query()
  - filter rows 


## 0. Load Libraries

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 0. Import Data

San Diego California is a popular vacation destination known for its beautiful beaches and warm weather. As a result, the city has a thriving AirBNB market with a wide variety of properties available for rent. According to data from Inside AirBNB, a company that tracks AirBNB data, San Diego had over 12,000 active listings as of December 2022. These listings included everything from private rooms to entire homes and apartments. 

Here we are going to import the listing data from San Diego California AirBNBs. 

http://insideairbnb.com/san-diego/

```python
abnb = pd.read_csv("https://raw.githubusercontent.com/MonkeyWrenchGang/MGTPython/main/module_3/data/sd_listings.csv")
abnb.head()
```


In [3]:
abnb = pd.read_csv("https://raw.githubusercontent.com/MonkeyWrenchGang/MGTPython/main/module_3/data/sd_listings.csv")
abnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6,Stay in the 1st house EVER posted on Airbnb,29,Sara,NaN,North Hills,32.75522,-117.12873,Entire home/apt,371,2,153,2019-09-14,0.87,1,10,0,NaN
1,29967,"Great home, 10 min walk to Beach",129123,Michael,NaN,Pacific Beach,32.80751,-117.25760,Entire home/apt,310,4,89,2022-12-21,0.59,5,235,17,NaN
2,38245,Point Loma: Den downstairs,164137,Melinda,NaN,Roseville,32.74217,-117.21931,Private room,113,1,149,2022-07-24,1.00,3,357,3,NaN
3,54001,"La Jolla Garden Cottage: Blks to Ocn; 2Bdms, 1...",252692,Marsha,NaN,La Jolla,32.81301,-117.26856,Entire home/apt,258,5,301,2022-11-18,2.06,2,67,21,"tier_2, STR-05706L"
4,62274,"charming, colorful, close to beach",302986,Isabel,NaN,Pacific Beach,32.80583,-117.24244,Entire home/apt,103,1,763,2022-12-04,5.20,3,300,68,NaN


## Our Challenge 

The San Diego AirBNB market is a strong and profitable option for both hosts and guests looking for a comfortable and convenient way to experience the city. We have several questions to answer:

0. how many nulls by column 
  - by row? 
1. what is the average daily rate for a San Diego AirBNB?
2. what areas have the highest prices? 
3. how many  properties by area ? 
4. what is the count/min/max/mean  ... by city 
5. what is the count/min/max/mean  by ... 
6. filter for ... then what is the count/min/max/mean
  - La Jolla
  - Pacific Beach
7. filter for 
8. filter for ... 
9. multi-step 
10. multi-step 

 




## Info

In [4]:
abnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12781 entries, 0 to 12780
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12781 non-null  int64  
 1   name                            12781 non-null  object 
 2   host_id                         12781 non-null  int64  
 3   host_name                       12778 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   12781 non-null  object 
 6   latitude                        12781 non-null  float64
 7   longitude                       12781 non-null  float64
 8   room_type                       12781 non-null  object 
 9   price                           12781 non-null  int64  
 10  minimum_nights                  12781 non-null  int64  
 11  number_of_reviews               12781 non-null  int64  
 12  last_review                     

# 0. Count Nulls


---

How many nulls are their in each column and how many nulls are their in each row? Using the Pandas library, count the number of null values in a DataFrame by using the `isnull()` method followed by the `sum(axis=0 or axis=1)` method.

1. produce a result of nulls by column - are their columns that contain 100% null values? 
2. create a new column "null_row_count" - which is the count of the number of nulls for each row. For example if neighborhood and price were both null in a row the count woudl be 2. Then filter for rows with > 4 null. 

## What si the axis = parameter?: 
In Pandas, the **axis = parameter** is used to specify which axis you want to apply a certain operation to.

- `axis = 0` refers to the rows of the DataFrame and is the default value for most operations. When using axis=0, the operation is applied vertically to each column for each row. This will give you the null counts by column. 

- `axis = 1` refers to the columns of the DataFrame. When using axis=1, the operation is applied horizontally to each row for each column. This will give you the null counts for each row of data. 


Here's an example:

```python
# count null columns
df.isnull().sum(axis=0)

# count null rows
df.isnull().sum(axis=1)
```


In [7]:
# count nulls by column 
abnb.isnull().sum(axis=0)

id                                    0
name                                  0
host_id                               0
host_name                             3
neighbourhood_group               12781
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                        1868
reviews_per_month                  1868
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           11166
dtype: int64

In [11]:
# create a column "null_row_count" wich contains the number of nulls found in a row of data. 
abnb["null_row_count"] = abnb.isnull().sum(axis=1)
abnb.sort_values("null_row_count", ascending=False).query('null_row_count > 4')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,null_row_count
1591,14116320,Modern La Jolla Escape,66115585,NaN,NaN,La Jolla,32.83220,-117.25315,Entire home/apt,1647,1,0,NaN,NaN,3,0,0,NaN,5
1594,14116330,Villa Portofino,66115585,NaN,NaN,La Jolla,32.84609,-117.26016,Entire home/apt,3049,1,0,NaN,NaN,3,0,0,NaN,5
1596,14116332,Forever Views,66115585,NaN,NaN,La Jolla,32.82401,-117.24479,Entire home/apt,1708,1,0,NaN,NaN,3,0,0,NaN,5



## Count nulls by record



In [ ]:
abnb["missing_count"] = abnb.isnull().sum(axis=1)
abnb.sort_values(["missing_count"], ascending=False)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,missing_count
2323,49187614,Quiet Bent Creek Bungalow,299484090,NaN,NaN,28806,35.51649,-82.59417,Entire home/apt,140,2,0,NaN,NaN,1,259,4
2332,49284012,Charming Asheville Studio Escape,21039179,Nga,NaN,28804,35.63852,-82.60889,Entire home/apt,78,1,0,NaN,NaN,1,167,3
2185,47963676,"Asheville bungalow with firepit, 2 bedrooms",14900836,Johanna,NaN,28804,35.62768,-82.61232,Entire home/apt,145,2,0,NaN,NaN,2,309,3
2196,48076290,Stylish West Asheville Private Apt. (30 Day Min),10923194,Amanda,NaN,28806,35.56833,-82.59794,Entire home/apt,110,30,0,NaN,NaN,1,13,3
2194,48053587,NEW! Asheville Forest Home by Creek - 7 Mi to ...,151397468,Evolve Vacation Rental,NaN,28803,35.53879,-82.48183,Entire home/apt,189,2,0,NaN,NaN,22,339,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,23234938,"Welcoming guests in Asheville. King Beds,Hot ...",56436566,Amanda & Ben,NaN,28806,35.63189,-82.65029,Entire home/apt,163,1,261,2021-04-05,6.79,2,306,1
779,23216051,Live/Work/Walk Everywhere from Drover's Way House,1329460,Lynne,NaN,28801,35.60129,-82.55514,Entire home/apt,135,30,70,2021-01-01,1.81,3,224,1
778,23150406,The Flint Street House downtown Historic Montford,37152088,Micha,NaN,28801,35.60190,-82.55996,Entire home/apt,123,1,404,2021-04-18,10.40,1,270,1
777,23084647,Cozy Little Farmette Minutes to Downtown Ashev...,6898358,Andrea,NaN,28805,35.60417,-82.50820,Private room,89,2,30,2021-04-13,0.95,1,177,1


## Aggregate Series

In [ ]:
abnb["price"].max()

6429

### Using .aggregate() function

In [ ]:
abnb["price"].aggregate(['mean', 'median', 'min', 'max', 'count'])

mean       166.540077
median     124.000000
min         19.000000
max       6429.000000
count     2333.000000
Name: price, dtype: float64

### How many listings' price is greater than the mean? 

In [ ]:
mean_price = abnb["price"].mean()

res1 = abnb.query('price > @mean_price')
res1.shape[0]

764

## Aggregate Dataframe

without specifying data types we get some odd results, because you can't take a mean of a string!

In [ ]:
abnb.agg(['mean','min','max'])

,id,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,missing_count
mean,2.982349e+07,NaN,1.089884e+08,NaN,28793.894985,35.577558,-82.556552,NaN,166.540077,4.758251,81.254608,2.853414,6.709387,174.881269,1.177454
min,1.080610e+05,"""ARTISH"" ... VERY Modern, Zen, Five Star Apart...",4.785900e+04,NaN,28704.000000,35.423420,-82.691050,Entire home/apt,19.000000,1.000000,0.000000,0.010000,1.000000,0.000000,1.000000
max,4.928401e+07,《Prime Location ❤Dog Friendly❤ Games & Fast Wifi》,3.966209e+08,NaN,28806.000000,35.655390,-82.459420,Shared room,6429.000000,180.000000,1008.000000,15.830000,80.000000,365.000000,4.000000


### Numeric Only

select_dtypes()



In [ ]:
abnb.select_dtypes(include= ["float64", 'int64']).agg( ['mean','min','max', 'nunique'] )

,price,minimum_nights
mean,166.540077,4.758251
min,19.000000,1.000000
max,6429.000000,180.000000
nunique,397.000000,22.000000


### Specify the Columns w. [[ ]]


In [ ]:
abnb[["minimum_nights", "price", "number_of_reviews" ]].agg( ['mean','min','max'] )

,minimum_nights,price,number_of_reviews
mean,4.758251,166.540077,81.254608
min,1.000000,19.000000,0.000000
max,180.000000,6429.000000,1008.000000


### Specify the Columns w. a Dict

In [ ]:
abnb.agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ["mean", "median", "count"]})

,minimum_nights,price,number_of_reviews
count,NaN,NaN,2333.000000
max,180.000000,6429.000000,NaN
mean,4.758251,166.540077,81.254608
median,NaN,NaN,44.000000
min,1.000000,19.000000,NaN


In [ ]:
my_agg_dict = {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ["mean", "median", "count"]}

abnb.agg(my_agg_dict)       

,minimum_nights,price,number_of_reviews
count,NaN,NaN,2333.000000
max,180.000000,6429.000000,NaN
mean,4.758251,166.540077,81.254608
median,NaN,NaN,44.000000
min,1.000000,19.000000,NaN


## Dealing with Strings / Object Types

We typically want to count complete records and the number of unique levels for a category.

In [ ]:
abnb.select_dtypes(include= ["object"]).agg( ['count','nunique', ] )

,name,host_name,room_type,last_review
count,2333,2279,2333,2153
nunique,2320,854,4,310


## Group By Aggregation

In [ ]:
abnb.groupby(["room_type"])[["price"]].agg( ['mean','min', 'max'] )

price          
                       mean min   max
room_type                            
Entire home/apt  176.548825  19  6429
Hotel room       251.736842  84   808
Private room     115.508951  25  1893
Shared room       62.500000  33   250

In [ ]:
abnb.groupby(["room_type"]).agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ["mean", "median", "count"]} )

minimum_nights           ... number_of_reviews             
                          mean min  max  ...              mean median count
room_type                                ...                               
Entire home/apt       5.200522   1  180  ...         80.396345     44  1915
Hotel room            1.000000   1    1  ...         11.263158      5    19
Private room          2.843990   1   31  ...         88.795396     46   391
Shared room           1.375000   1    3  ...         84.375000     76     8

[4 rows x 9 columns]

In [ ]:
abnb.groupby(["room_type"]).agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ["mean", 'min','max']} ).stack().reset_index()

,room_type,level_1,minimum_nights,price,number_of_reviews
0,Entire home/apt,mean,5.200522,176.548825,80.396345
1,Entire home/apt,min,1.000000,19.000000,0.000000
2,Entire home/apt,max,180.000000,6429.000000,717.000000
3,Hotel room,mean,1.000000,251.736842,11.263158
4,Hotel room,min,1.000000,84.000000,0.000000
5,Hotel room,max,1.000000,808.000000,54.000000
6,Private room,mean,2.843990,115.508951,88.795396
7,Private room,min,1.000000,25.000000,0.000000
8,Private room,max,31.000000,1893.000000,1008.000000
9,Shared room,mean,1.375000,62.500000,84.375000


In [ ]:
abnb.groupby(["room_type"]).agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ['mean','min','max']} ).stack().reset_index()

,room_type,level_1,minimum_nights,price,number_of_reviews
0,Entire home/apt,mean,5.200522,176.548825,80.396345
1,Entire home/apt,min,1.000000,19.000000,0.000000
2,Entire home/apt,max,180.000000,6429.000000,717.000000
3,Hotel room,mean,1.000000,251.736842,11.263158
4,Hotel room,min,1.000000,84.000000,0.000000
5,Hotel room,max,1.000000,808.000000,54.000000
6,Private room,mean,2.843990,115.508951,88.795396
7,Private room,min,1.000000,25.000000,0.000000
8,Private room,max,31.000000,1893.000000,1008.000000
9,Shared room,mean,1.375000,62.500000,84.375000


## Multi-Level Group by 

In [ ]:
abnb.groupby(["neighbourhood","room_type"]).agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ['mean','min','max']} )

minimum_nights      ... number_of_reviews      
                                        mean min  ...               min   max
neighbourhood room_type                           ...                        
28704         Entire home/apt       2.230088   1  ...                 0   395
              Private room          7.388889   1  ...                 6   366
28715         Entire home/apt       3.034483   1  ...                 0   443
              Private room          1.666667   1  ...                19    87
28732         Entire home/apt       2.509804   1  ...                 0   234
              Private room          1.250000   1  ...                 0   193
28801         Entire home/apt       8.051402   1  ...                 0   707
              Hotel room            1.000000   1  ...                 0    23
              Private room          2.682171   1  ...                 0  1008
              Shared room           1.333333   1  ...                 3    81
28803         Entire home/apt       5.297491   1  ...                 0   554
              Hotel room            1.000000   1  ...                31    54
              Private room          3.041667   1  ...                 0   866
28804         Entire home/apt       5.067460   1  ...                 0   514
              Private room          2.785714   1  ...                 0   442
              Shared room           3.000000   3  ...                 3     3
28805         Entire home/apt       4.427184   1  ...                 0   558
              Private room          2.591837   1  ...                 0   523
28806         Entire home/apt       4.337121   1  ...                 0   717
              Private room          2.495238   1  ...                 0   558
              Shared room           1.000000   1  ...                71   282

[21 rows x 9 columns]

## Filtering w. Group by and Aggregate

My recomendation is to filter rows up front using query

In [ ]:
abnb.query('room_type == "Private room"').groupby(["neighbourhood","room_type"]).agg( {"minimum_nights": ['mean','min','max'], 
           "price" : ['mean','min','max'], 
           "number_of_reviews" : ['mean','min','max']} )

minimum_nights          ... number_of_reviews          
                                     mean min max  ...              mean min   max
neighbourhood room_type                            ...                            
28704         Private room       7.388889   1  30  ...         84.500000   6   366
28715         Private room       1.666667   1   2  ...         64.833333  19    87
28732         Private room       1.250000   1   2  ...         89.250000   0   193
28801         Private room       2.682171   1  30  ...         89.984496   0  1008
28803         Private room       3.041667   1  31  ...        108.687500   0   866
28804         Private room       2.785714   1  30  ...         68.857143   0   442
28805         Private room       2.591837   1  30  ...         96.938776   0   523
28806         Private room       2.495238   1  30  ...         81.828571   0   558

[8 rows x 9 columns]

In [ ]:
## -- Method Chaining / pipelining style -- 

RES1 = ( abnb
        .query('room_type == "Private room"')
        .groupby(["neighbourhood","room_type"])
        .agg( {"minimum_nights": ['mean','min','max'], 
               "price" : ['mean','min','max'], 
               "number_of_reviews" : ['mean','min','max']} )
        .stack()
        .reset_index()
        .query('neighbourhood == "28704" ')
)[["neighbourhood", "level_2", "minimum_nights"]]

RES1

,neighbourhood,level_2,minimum_nights
0,28704,mean,7.388889
1,28704,min,1.000000
2,28704,max,30.000000
